In [1]:
import pandas as pd
import numpy as np
import math
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


In [2]:
#quarter features
feature_1 = pd.read_csv("data_set_1.csv")
feature_2 = pd.read_csv("data_set_2.csv")
feature_3 = pd.read_csv("data_set_3.csv")
feature_4 = pd.read_csv("data_set_4.csv")
feature_5 = pd.read_csv("data_set_5.csv")
feature_6 = pd.read_csv("data_set_6.csv")
feature_7 = pd.read_csv("data_set_7.csv")
#daily features
feature_8 = pd.read_csv("data_set_8.csv")
feature_9 = pd.read_csv("data_set_9.csv")
feature_10 = pd.read_csv("data_set_10.csv")
feature_11 = pd.read_csv("data_set_11.csv")
#daily return
stock_rtn_1 = pd.read_csv("security_reference_data_w_ret1d_1.csv")
stock_rtn_2 = pd.read_csv("security_reference_data_w_ret1d_2.csv")
#risk_factors
risk_factors_1 = pd.read_csv("risk_factors_1.csv")
risk_factors_2 = pd.read_csv("risk_factors_2.csv")
#merge daily retruns
stock_rtn = pd.concat([stock_rtn_1,stock_rtn_2],axis = 0)
stock_rtn.to_csv("stock_rtn.csv")

# Quarterly cross-section factor 

In [40]:
def cal_stock_rtn(stock_id,time_period = 5):
    stock_temp = stock_rtn[stock_rtn.security_id == stock_id]
    stock_temp["log_ret_1"] = stock_temp["ret1d"].apply(lambda x:math.log(1+x)).shift(-1)
    stock_temp["log_ret_1"].rolling(window = time_period).sum().shift(-time_period+1)
    return stock_temp["log_ret_1"].rolling(window = time_period).sum().shift(-time_period+1).fillna(0)

def cal_stock_rtn_diff_period(stock_id):
    stock_temp = stock_rtn[stock_rtn.security_id == stock_id]
    stock_temp["log_ret_1"] = cal_stock_rtn(stock_id,1)
    stock_temp["log_ret_5"] = cal_stock_rtn(stock_id,5)
    stock_temp["log_ret_10"] = cal_stock_rtn(stock_id,10)
    stock_temp["log_ret_22"] = cal_stock_rtn(stock_id,22)
    stock_temp["log_ret_66"] = cal_stock_rtn(stock_id,66)
    return stock_temp[["data_date","security_id","log_ret_1","log_ret_5","log_ret_10","log_ret_22","log_ret_66"]]

def cal_all_stock_rtn():
    stock_list = list(set(stock_rtn["security_id"]))
    print(len(stock_list))
    df_init = cal_stock_rtn_diff_period(stock_list[0])
    for i in range(1,len(stock_list)):
        df_temp = cal_stock_rtn_diff_period(stock_list[i])
        df_init = pd.concat([df_init,df_temp],axis = 0)
        
    return df_init


df = cal_all_stock_rtn()
df.to_csv("rtn_diff_period.csv")

7876


In [4]:
def to_quarter(dataframe):
    def tranfer(string):
        if string in ["01","02","03"]:
            return 1
        if string in ["04","05","06"]:
            return 2
        if string in ["07","08","09"]:
            return 3
        if string in ["10","11","12"]:
            return 4
    dataframe["year"] = dataframe["data_date"].apply(lambda x: int(str(x)[:4]))
    dataframe["quarter"] = dataframe["data_date"].apply(lambda x: tranfer(str(x)[4:6]))
    dataframe["year"] = dataframe["year"].apply(lambda x: str(x))
    dataframe["quarter"] = dataframe["quarter"].apply(lambda x: "q"+str(x))
    dataframe["quarter"]  = dataframe["year"] + dataframe["quarter"] 
    return dataframe

def cal_rtn(dataframe):
    df = pd.read_csv("rtn_diff_period.csv")
    dataframe = to_quarter(dataframe)
    df_init = pd.DataFrame({"log_ret_1":[0],"log_ret_5":[0],"log_ret_10":[0],"log_ret_22":[0],"log_ret_66":[0]})
    df_new = df_init
    for i in range(len(dataframe)):
        df_temp = df[(df.security_id == dataframe["security_id"][i])&(df.data_date == dataframe["data_date"][i])][["log_ret_1","log_ret_5","log_ret_10","log_ret_22","log_ret_66"]]
        if abs(df_temp.values).sum() !=0:
            df_new = pd.concat([df_new,df_temp])
        else:
            df_new = pd.concat([df_new,df_init])
    
    df_new = df_new[1:]
    df_new.index = range(len(df_new))
    df_new = pd.concat([dataframe,df_new],axis =1)

    
    
    
    return df_new



In [42]:
stock_rtn = pd.read_csv("stock_rtn.csv")
rtn_diff_period = pd.read_csv("rtn_diff_period.csv")

In [43]:
rtn_diff_period = rtn_diff_period[["data_date","security_id","log_ret_1"]]

In [44]:
rtn_diff_period

,data_date,security_id,log_ret_1
0,20100104,4915201,-0.003960
1,20100105,4915201,0.038915
2,20100106,4915201,0.005709
3,20100107,4915201,0.009443
4,20100108,4915201,0.022306
...,...,...,...
8800010,20171222,1115914201,-0.018454
8800011,20171226,1115914201,0.016124
8800012,20171227,1115914201,-0.016124
8800013,20171228,1115914201,-0.007137


In [45]:
stock_rtn = pd.merge(stock_rtn,rtn_diff_period,on = ["data_date","security_id"],how = "left")

In [47]:
stock_rtn.to_csv("stock_rtn.csv")

add future retruns to the feature list

In [5]:
feature_1 = feature_1.dropna()
feature_1 = feature_1.drop_duplicates()
feature_1 = to_quarter(feature_1)
df = pd.read_csv("rtn_diff_period.csv")[["data_date","security_id","log_ret_66"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_1,df,on = ["data_date","security_id"],how = "left")

df_new.to_csv("feature_1.csv")

In [6]:
feature_2 = feature_2.dropna()
feature_2 = feature_2.drop_duplicates()
feature_2 = to_quarter(feature_2)
df = pd.read_csv("rtn_diff_period.csv")[["data_date","security_id","log_ret_66"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_2,df,on = ["data_date","security_id"],how = "left")
df_new.to_csv("feature_2.csv")

In [7]:
feature_3 = feature_3.dropna()
feature_3 = feature_3.drop_duplicates()
feature_3 = to_quarter(feature_3)
df = pd.read_csv("rtn_diff_period.csv")[["data_date","security_id","log_ret_66"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_3,df,on = ["data_date","security_id"],how = "left")
df_new.to_csv("feature_3.csv")

In [8]:
feature_4 = feature_4.dropna()
feature_4 = feature_4.drop_duplicates()
feature_4 = to_quarter(feature_4)
df = pd.read_csv("rtn_diff_period.csv")[["data_date","security_id","log_ret_66"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_4,df,on = ["data_date","security_id"],how = "left")
df_new.to_csv("feature_4.csv")


In [9]:
feature_5 = feature_5.dropna()
feature_5 = feature_5.drop_duplicates()
feature_5 = to_quarter(feature_5)
df = pd.read_csv("rtn_diff_period.csv")[["data_date","security_id","log_ret_66"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_5,df,on = ["data_date","security_id"],how = "left")
df_new.to_csv("feature_5.csv")

feature_6 = feature_6.dropna()
feature_6 = feature_6.drop_duplicates()
feature_6 = to_quarter(feature_6)
df = pd.read_csv("rtn_diff_period.csv")[["data_date","security_id","log_ret_66"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_6,df,on = ["data_date","security_id"],how = "left")
df_new.to_csv("feature_6.csv")

feature_7 = feature_7.dropna()
feature_7 = feature_7.drop_duplicates()
feature_7 = to_quarter(feature_7)
df = pd.read_csv("rtn_diff_period.csv")[["data_date","security_id","log_ret_66"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_7,df,on = ["data_date","security_id"],how = "left")
df_new.to_csv("feature_7.csv")


# Daily Time-Series factor

In [10]:
stock_rtn = pd.read_csv("stock_rtn.csv")

In [11]:
def cal_stock_rtn(stock_id,time_period = 5):
    stock_temp = stock_rtn[stock_rtn.security_id == stock_id]
    stock_temp["log_ret_1"] = stock_temp["ret1d"].apply(lambda x:math.log(1+x)).shift(-1)
    return stock_temp["log_ret_1"].rolling(window = time_period).sum().shift(-time_period+1).fillna(0)

def cal_stock_rtn_diff_period(stock_id):
    stock_temp = stock_rtn[stock_rtn.security_id == stock_id]
    stock_temp["log_ret_1"] = cal_stock_rtn(stock_id,1)

    return stock_temp[["data_date","security_id","log_ret_1"]]

def cal_all_stock_rtn():
    stock_list = list(set(stock_rtn["security_id"]))
    df_init = cal_stock_rtn_diff_period(stock_list[0])
    for i in range(1,len(stock_list)):
        df_temp = cal_stock_rtn_diff_period(stock_list[i])
        df_init = pd.concat([df_init,df_temp],axis = 0)
        
    return df_init


df = cal_all_stock_rtn()
df
df.to_csv("rtn_diff_short_period.csv")

add return to the feature

In [12]:
feature_8 = feature_8.dropna()
feature_8 = feature_8.drop_duplicates()
df = pd.read_csv("rtn_diff_short_period.csv")[["data_date","security_id","log_ret_1"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_8,df,on = ["data_date","security_id"],how = "left")
df_new.fillna(0,inplace = True)
df_new.to_csv("feature_8.csv")

In [13]:
feature_9 = feature_9.dropna()
feature_9 = feature_9.drop_duplicates()
df = pd.read_csv("rtn_diff_short_period.csv")[["data_date","security_id","log_ret_1"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_9,df,on = ["data_date","security_id"],how = "left")
df_new.fillna(0,inplace = True)
df_new.to_csv("feature_9.csv")

In [14]:
feature_10 = feature_10.dropna()
feature_10 = feature_10.drop_duplicates()
df = pd.read_csv("rtn_diff_short_period.csv")[["data_date","security_id","log_ret_1"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_10,df ,on = ["data_date","security_id"],how = "left")
df_new.fillna(0,inplace = True)
df_new.to_csv("feature_10.csv")

In [15]:
feature_11 = feature_11.dropna()
feature_11 = feature_11.drop_duplicates()
df = pd.read_csv("rtn_diff_short_period.csv")[["data_date","security_id","log_ret_1"]]
df.fillna(0,inplace = True)
df_new = pd.merge(feature_11,df ,on = ["data_date","security_id"],how = "left")
df_new.fillna(0,inplace = True)
df_new.to_csv("feature_11.csv")

In [17]:
stock_rtn = pd.read_csv("stock_rtn.csv")

In [19]:
stock_rtn[stock_rtn.security_id == 4915201]

,Unnamed: 0,data_date,security_id,close_price,volume,group_id,in_trading_universe,ret1d
2452,2452,20100104,4915201,5.0600,146389.0,35201010,N,0.032653
7134,7134,20100105,4915201,5.0400,116322.0,35201010,N,-0.003953
11802,11802,20100106,4915201,5.2400,605047.0,35201010,N,0.039683
16465,16465,20100107,4915201,5.2700,235562.0,35201010,N,0.005725
21126,21126,20100108,4915201,5.3200,379861.0,35201010,N,0.009488
...,...,...,...,...,...,...,...,...
7799840,3488165,20170207,4915201,1.1399,39308.0,35201010,N,0.026937
7804275,3492600,20170208,4915201,1.1200,196692.0,35201010,N,-0.017458
7808713,3497038,20170209,4915201,1.1200,40592.0,35201010,N,0.000000
7813165,3501490,20170210,4915201,1.1400,166746.0,35201010,N,0.017857
